In [1]:
import pandas as pd
import numpy as np

### train

In [20]:
train = pd.read_csv('../input/train.csv')
train.head()

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1


### test

In [21]:
test = pd.read_csv('../input/sample_submission_zero.csv')
test.drop('is_churn',axis=1,inplace=True)
test.head()

,msno
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=


### members

In [4]:
members = pd.read_csv('../input/members.csv')
members.replace('female',0,inplace=True)
members.replace('male',1,inplace=True)
members.fillna(2,inplace=True)
members.gender = members.gender.astype('int')
members['exp_to_reg_interval'] = (pd.to_datetime(members.expiration_date,format='%Y%m%d')-pd.to_datetime(members.registration_init_time,format='%Y%m%d')).map(lambda x:x.days)
members.drop(['registration_init_time','expiration_date'],axis=1,inplace=True)
members.head()

,msno,city,bd,gender,registered_via,exp_to_reg_interval
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,2,9,1
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,2,4,3
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,2,4,3
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,0,4,7
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,2,4,3


In [5]:
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116194 entries, 0 to 5116193
Data columns (total 6 columns):
msno                   object
city                   int64
bd                     int64
gender                 int32
registered_via         int64
exp_to_reg_interval    int64
dtypes: int32(1), int64(4), object(1)
memory usage: 214.7+ MB


## transactions

计种类数是一种方法 还有种方法是离散 不过效果就不能保证了

In [6]:
transactions = pd.read_csv('../input/transactions.csv')

#统计个数（非离散累加）
#用户使用的支付方式的种类个数（暂时不离散） 可推广至各个
transactions['transactions_count'] = 1
t0 = transactions[['msno','transactions_count']].groupby('msno',as_index=False).sum()
t1 = transactions[['msno','payment_method_id']].groupby('msno',as_index=False).agg(lambda x:len(np.unique(x)))
t1.columns = ['msno','payment_method_id_cate_count']
t2 = transactions[['msno','payment_plan_days']].groupby('msno',as_index=False).agg(lambda x:len(np.unique(x)))
t2.columns = ['msno','payment_plan_days_cate_count']
t3 = transactions[['msno','plan_list_price']].groupby('msno',as_index=False).agg(lambda x:len(np.unique(x)))
t3.columns = ['msno','plan_list_price_cate_count']
t4 = transactions[['msno','actual_amount_paid']].groupby('msno',as_index=False).agg(lambda x:len(np.unique(x)))
t4.columns = ['msno','actual_amount_paid_cate_count']
t5 = transactions[['msno','is_auto_renew']].groupby('msno',as_index=False).agg('mean')
t5.columns = ['msno','auto_renew_rate']
t6 = transactions[['msno','is_cancel']].groupby('msno',as_index=False).agg('mean')
t6.columns = ['msno','cancel_rate']
t7 = transactions[['msno','payment_plan_days']].groupby('msno',as_index=False).agg('mean')
t7.columns = ['msno','payment_plan_days_mean']
t8 = transactions[['msno','plan_list_price']].groupby('msno',as_index=False).agg('mean')
t8.columns = ['msno','plan_list_price_mean']


##假定订阅日期和订阅时长重复 那就先删了吧
#transactions.drop(['transaction_date','membership_expire_date'],axis=1,inplace=True)

#合并
transactions = t0
transactions = pd.merge(transactions,t1,on='msno',how='left')
transactions = pd.merge(transactions,t2,on='msno',how='left')
transactions = pd.merge(transactions,t3,on='msno',how='left')
transactions = pd.merge(transactions,t4,on='msno',how='left')
transactions = pd.merge(transactions,t5,on='msno',how='left')
transactions = pd.merge(transactions,t6,on='msno',how='left')
transactions = pd.merge(transactions,t7,on='msno',how='left')
transactions = pd.merge(transactions,t8,on='msno',how='left')

del t0,t1,t2,t3,t4,t5,t6,t7,t8
transactions.head()

,msno,transactions_count,payment_method_id_cate_count,payment_plan_days_cate_count,plan_list_price_cate_count,actual_amount_paid_cate_count,auto_renew_rate,cancel_rate,payment_plan_days_mean,plan_list_price_mean
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,1,1,1,1,0.0,0.0,7.000000,0.000000
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,1,1,1,1,0.0,0.0,410.000000,1788.000000
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,1,1,1,1,1.0,0.0,30.000000,99.000000
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,1,3,2,1,1.0,0.0,28.578947,141.157895
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,1,2,2,1,1.0,0.0,28.846154,143.269231


### user_logs

大文件 分块肝 设计的特征尽量少读

In [7]:
user_logs = None

reader = pd.read_csv('../input/user_logs.csv',iterator=True)
loop = True
chunkSize = 50000000 #根据自己电脑修改
i = 0
while loop:
    print(i)
    i+=1
    try:
        chunk = reader.get_chunk(chunkSize)
        chunk.drop('date',axis=1,inplace=True)
        chunk['total_hours'] = chunk['total_secs'] / 3600 #小时
        chunk.drop('total_secs',axis=1,inplace=True)
        chunk['user_log_count'] = 1
        chunk = chunk.groupby('msno',as_index=False).sum()
        if user_logs is None:
            user_logs = chunk
        else:
            user_logs = pd.concat([user_logs,chunk],axis=0)
            user_logs = user_logs.groupby('msno',as_index=False).sum()
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
user_logs['mean_hours'] = user_logs['total_hours'] / user_logs['user_log_count']
user_logs.head()

0
1
2
3
4
5
6
7
8
Iteration is stopped.


,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_hours,user_log_count,mean_hours
0,+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,1,1,0,0,0,2,0.027059,1,0.027059
1,+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,2,0,0,0,0,1,0.015797,1,0.015797
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,60,12,14,7,171,179,13.887992,7,1.983999
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,817,249,227,195,59354,53604,4073.567057,622,6.549143
4,+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,3,3,2,1,14,22,1.017091,1,1.017091


In [8]:
user_logs.head()

,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_hours,user_log_count,mean_hours
0,+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,1,1,0,0,0,2,0.027059,1,0.027059
1,+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,2,0,0,0,0,1,0.015797,1,0.015797
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,60,12,14,7,171,179,13.887992,7,1.983999
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,817,249,227,195,59354,53604,4073.567057,622,6.549143
4,+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,3,3,2,1,14,22,1.017091,1,1.017091


## GENERATE TRAIN AND TEST SET

In [22]:
train = pd.merge(train,members,on='msno',how='left')
train = pd.merge(train,transactions,on='msno',how='left')
train = pd.merge(train,user_logs,on='msno',how='left')
train.head()

,msno,is_churn,city,bd,gender,registered_via,exp_to_reg_interval,transactions_count,payment_method_id_cate_count,payment_plan_days_cate_count,...,plan_list_price_mean,num_25,num_50,num_75,num_985,num_100,num_unq,total_hours,user_log_count,mean_hours
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,0.0,9.0,4537.0,2,1,2,...,74.500000,51.0,16.0,19.0,11.0,409.0,462.0,30.014404,26.0,1.154400
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,1.0,9.0,4366.0,23,2,3,...,143.869565,447.0,157.0,121.0,178.0,9942.0,7232.0,726.475594,521.0,1.394387
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,0.0,9.0,4128.0,10,2,1,...,149.200000,1570.0,950.0,302.0,479.0,10033.0,11533.0,721.495738,237.0,3.044286
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,0.0,9.0,4346.0,2,2,1,...,894.000000,7265.0,2420.0,1789.0,1634.0,18070.0,18479.0,1373.696905,735.0,1.868975
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,1.0,9.0,4291.0,8,1,3,...,447.000000,1389.0,714.0,682.0,3228.0,68731.0,72674.0,5178.929723,758.0,6.832361


In [24]:
train.to_csv('../data/train.csv',index=None)

In [23]:
test = pd.merge(test,members,on='msno',how='left')
test = pd.merge(test,transactions,on='msno',how='left')
test = pd.merge(test,user_logs,on='msno',how='left')
test.head()

,msno,city,bd,gender,registered_via,exp_to_reg_interval,transactions_count,payment_method_id_cate_count,payment_plan_days_cate_count,plan_list_price_cate_count,...,plan_list_price_mean,num_25,num_50,num_75,num_985,num_100,num_unq,total_hours,user_log_count,mean_hours
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,5.0,28.0,1.0,3.0,1338.0,6.0,2.0,2.0,2.0,...,49.666667,367.0,84.0,44.0,82.0,1713.0,430.0,111.193170,53.0,2.097984
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,13.0,20.0,1.0,3.0,1243.0,5.0,3.0,2.0,3.0,...,125.400000,147.0,41.0,21.0,25.0,782.0,703.0,58.343537,74.0,0.788426
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13.0,18.0,1.0,3.0,1362.0,18.0,1.0,1.0,1.0,...,149.000000,10545.0,724.0,391.0,399.0,6481.0,14866.0,517.788346,322.0,1.608038
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,0.0,2.0,7.0,806.0,19.0,2.0,3.0,2.0,...,125.473684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13.0,35.0,0.0,7.0,1485.0,24.0,1.0,2.0,4.0,...,105.375000,308.0,187.0,115.0,71.0,3091.0,3359.0,225.049045,256.0,0.879098


In [25]:
test.to_csv('../data/test.csv',index=None)